In [1]:
from sklearn.neighbors import KNeighborsClassifier
import pickle
import pandas as pd

In [2]:
def loadPickle(name):
    file = open(name, 'rb')
    data = pickle.load(file)
    file.close()
    
    return data

In [3]:
def listOfDictToDataFrame(listOfDict):
    dataFrame = pd.DataFrame(listOfDict)
    
    x = dataFrame.iloc[:, :-1].values
    y = dataFrame.iloc[:, 4].values
    
    return dataFrame, x, y

In [4]:
data = loadPickle("training.pickle")
dataFrame, x, y = listOfDictToDataFrame(data)

In [6]:
dataFrame.head()

,index,first_upper_case,prev_word,word,jenis
0,0,y,null,Menteri,o
1,1,y,Menteri,Pertahanan,o
2,3,y,Pertahanan,Prabowo,person
3,4,y,Prabowo,Subianto,person
4,6,n,Subianto,dikabarkan,o


In [7]:
#biar string jadi skala
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(x)
enc.categories_
x = enc.transform(x).toarray()

In [8]:
#biar string jadi skala
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
list(le.classes_)
y = le.transform(y)

In [9]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [12]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words = line.split(' ') #tiap satu baris split per kata
    
    f.close()
    
    words[len(words)-1] = words[len(words)-1].replace('\n',' ')

    return words

In [15]:
y_pred = classifier.predict(x)

In [16]:
print(y_pred)

[1 1 3 3 1 1 1 3 3 1 1 1 1 3 1 1 1 1 1 3 3 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 2 2 1 2 2 2 1 1 2 1 3 2 3 1 1 1 1 1 1 3 3 1 1 1 1 1 2 2 2 2 1 1 1 3 1 1 1
 2 1 3 1 1 1 2 1 3 1 1 1 1 1 1 2 3 1 1 1 1 3 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 3 1]


In [ ]:
list(le.inverse_transform(y_pred))